In [ ]:
from google.colab import drive
drive.mount('/content/drive') 

Mounted at /content/drive


In [ ]:
import keras
import tensorflow as tf
import numpy as np
import pandas as pd
import re
import tensorflow
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,SimpleRNN,Dense, Dropout, SpatialDropout1D
from tensorflow.keras.layers import Embedding
from keras.layers import Embedding

In [ ]:
#define train and test files
TRAIN_PATH = "/content/drive/MyDrive/fallabella/sentiments_train_test_reviews/reviews_training_26000.csv"
TEST_PATH = "/content/drive/MyDrive/fallabella/sentiments_train_test_reviews/reviews_test_4000.csv"

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
df_train = pd.read_csv(TRAIN_PATH)
df_test = pd.read_csv(TEST_PATH)

In [ ]:
df_train.shape

(26005, 3)

In [ ]:
df_train.head()

,review_id,review,sentiment
0,2,I thought this was a wonderful way to spend ti...,positive
1,3,Basically there's a family where a little boy ...,negative
2,7,"This show was an amazing, fresh & innovative i...",negative
3,8,Encouraged by the positive comments about this...,negative
4,10,Phil the Alien is one of those quirky films wh...,negative


In [ ]:
df_test.shape

(4000, 3)

In [ ]:
df_train = df_train[['review','sentiment']]
df_test = df_test[['review','sentiment']]

In [ ]:
df_test.head(4)

,review,sentiment
0,I have to confess that I am severely disappoin...,negative
1,I have never understood the appeal of this sho...,negative
2,This is supposed to be based on Wilkie Collins...,negative
3,Of all the British imperialist movies like Fou...,positive


In [ ]:
df_train.value_counts(['sentiment'])

sentiment
negative     20007
positive      5998
dtype: int64

In [ ]:
df_train['review'].isnull().sum()

0

In [ ]:
df_train['sentiment'] = df_train['sentiment'].replace({"positive":1,"negative":0})
df_test['sentiment'] = df_train['sentiment'].replace({"positive":1,"negative":0})

In [ ]:
df_train.head()

,review,sentiment
0,I thought this was a wonderful way to spend ti...,1
1,Basically there's a family where a little boy ...,0
2,"This show was an amazing, fresh & innovative i...",0
3,Encouraged by the positive comments about this...,0
4,Phil the Alien is one of those quirky films wh...,0


In [ ]:
df_train['review'] = df_train['review'].str.lower()

In [ ]:
df_train['review'] = df_train['review'].str.replace(r'http:\S+','',regex = True)
df_train['review'] = df_train['review'].str.replace(r'A-Za-z0-9','',regex = True)
df_train['review'] = df_train['review'].apply(lambda words:' '.join(word.lower() for word in words.split() if word not in stopwords.words('english')))



In [ ]:
lemmatizer = WordNetLemmatizer()

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [ ]:
def tokenization_with_lemma(sentences):
  for i in range(len(sentences)):
    words = nltk.word_tokenize(sentences[i])
    words = re.sub('[^a-zA-Z]', ' ', sentences[i])
    words = [word.lower() for word in words]
    lemma_words = [lemmatizer.lemmatize(word) for word in words if word not in set(stopwords.words('english'))]
    sentences[i] = ''.join(lemma_words)
  return sentences

In [ ]:
df_train.head(2)

,review,sentiment
0,i thought this was a wonderful way to spend ti...,1
1,basically there's a family where a little boy ...,0


In [ ]:
train_review = df_train['review'].tolist()
test_review = df_test['review'].tolist()

In [ ]:
df_train['review'] = tokenization_with_lemma(train_review)
df_test['review'] = tokenization_with_lemma(test_review)


In [ ]:
result = [len(x) for x in df_train['review']]
print("Mean %.2f words (%f)" % (np.mean(result), np.std(result)))

Mean 1291.70 words (950.707228)


In [ ]:
train_review = df_train['review'].values
test_review	 = df_train['review'].values

In [ ]:
tokenizer = Tokenizer(num_words=50000)

tokenizer.fit_on_texts(train_review)
tokenizer.fit_on_texts(test_review)


In [ ]:
vocab_size = len(tokenizer.word_index) + 1

In [ ]:
vocab_size

90010

In [ ]:
encoded_docs_train = tokenizer.texts_to_sequences(train_review)
encoded_docs_test = tokenizer.texts_to_sequences(test_review)

In [ ]:
padded_sequence_train = pad_sequences(encoded_docs_train, maxlen=1500)
padded_sequence_test = pad_sequences(encoded_docs_test, maxlen=1500)

In [ ]:
embedding_vector_length = 32
model = Sequential()
model.add(Embedding(vocab_size, embedding_vector_length, input_length=1500))
model.add(SpatialDropout1D(0.25))
model.add(SimpleRNN(512, return_sequences=True,activation = 'relu'))
model.add(Dropout(0.2))
model.add(SimpleRNN(256, return_sequences=True,activation = 'relu'))
model.add(Dropout(0.2))
model.add(SimpleRNN(128, return_sequences=True,activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 1500, 32)          2880320   
                                                                 
 spatial_dropout1d (SpatialD  (None, 1500, 32)         0         
 ropout1D)                                                       
                                                                 
 simple_rnn (SimpleRNN)      (None, 1500, 512)         279040    
                                                                 
 dropout (Dropout)           (None, 1500, 512)         0         
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 1500, 256)         196864    
                                                                 
 dropout_1 (Dropout)         (None, 1500, 256)         0         
                                                        

In [ ]:
padded_sequence_train.shape

(26005, 1500)

In [ ]:
hr_title = model.fit(padded_sequence_train,df_train['sentiment'], epochs=15, batch_size=128)

Epoch 1/15
204/204 [==============================] - 3626s 18s/step - loss: nan - accuracy: 0.7680
Epoch 2/15
202/204 [============================>.] - ETA: 35s - loss: nan - accuracy: 0.7691